<a href="https://colab.research.google.com/github/hugotomita1201/yachay.ai_project/blob/main/electra_model_ungrouped_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

# %%
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import time
from transformers import AutoTokenizer
from transformers import AutoModel
import torch.nn as nn
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def haversine_distance(lat1, lon1, lat2, lon2):
    earth_radius = 6371  # Earth radius in km
    pi = torch.tensor(3.141592653589793, dtype=torch.float)

    lat1, lon1, lat2, lon2 = [x * (pi / 180) for x in [lat1, lon1, lat2, lon2]]

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = torch.sin(dlat / 2) ** 2 + torch.cos(lat1) * \
        torch.cos(lat2) * torch.sin(dlon / 2) ** 2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))

    distance = earth_radius * c
    return distance


class HaversineLoss(nn.Module):
    def __init__(self):
        super(HaversineLoss, self).__init__()

    def forward(self, y_pred, y_true):
        lat1, lon1 = y_pred[:, 0], y_pred[:, 1]
        lat2, lon2 = y_true[:, 0], y_true[:, 1]

        distance = haversine_distance(lat1, lon1, lat2, lon2)
        loss = distance.mean()
        return loss


haversine_loss = HaversineLoss()
# Replace the existing loss function in the training loop with haversine_loss
loss_fn = haversine_loss


# %%
# Train the model
num_epochs = 10
batch_size = 64


class GlobalScaler():
    def __init__(self):
        self.means = None
        self.stddev = None

    def fit_transform(self, data):
        self.means = np.mean(data, axis=0)
        centereddata = data - self.means
        self.stddev = np.std(centereddata)
        return centereddata / self.stddev

    def transform(self, data):
        return (data - self.means) / self.stddev

    def inverse_transform(self, data):
        return (data * self.stddev) + self.means


# %%
df = pd.read_csv('Clusters_Coordinates_Language_Labeled.csv')


# split the data into train and test
tokenizer = AutoTokenizer.from_pretrained(
    "google/electra-small-discriminator", do_lower_case=True)
model = AutoModel.from_pretrained(
    'google/electra-small-discriminator', num_labels=2).to(device)
config = model.config


# split data into train and test


def preprocess_data(df, tokenizer):

    # load start time
    start_time = time.time()
    # print that the data is being loaded
    print("Loading data...")

    # Filter and sample data
    df = df[df['language'] == 0]

    # Split into train and test sets
    train_features, test_features, train_labels, test_labels = train_test_split(
        df['text'], df[['lat', 'lng']], test_size=0.25, random_state=42)

    # Load tokenizer
    tokenizer = tokenizer

    # Tokenize the training and test text features
    train_encodings = tokenizer(train_features.tolist(
    ), padding=True, truncation=True, return_tensors='pt')
    test_encodings = tokenizer(test_features.tolist(
    ), padding=True, truncation=True, return_tensors='pt')

    # Tokenize the training and test text features
    train_input_ids = train_encodings['input_ids']
    test_input_ids = test_encodings['input_ids']

    # make attention masks to let know which tokens are real and which are padding
    train_attention_mask = train_encodings['attention_mask']
    test_attention_mask = test_encodings['attention_mask']

    # Convert labels to PyTorch tensors with float datatype
    train_labels = torch.tensor(
        train_labels[['lat', 'lng']].values, dtype=torch.float)
    test_labels = torch.tensor(
        test_labels[['lat', 'lng']].values, dtype=torch.float)

    # Print the time taken to load the data
    print("Time taken to load data: {} seconds".format(
        time.time() - start_time))

    return train_encodings, test_encodings, train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels


train_encodings, test_encodings, train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = preprocess_data(
    df, tokenizer)


# Define the BERT-based reg model
class BERTRegressor(nn.Module):
    def __init__(self, model, num_inputs, num_outputs):
        super(BERTRegressor, self).__init__()
        self.bert = model
        self.dropout = nn.Dropout(0.1)
        self.regressor = nn.Linear(num_inputs, num_outputs)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = bert_outputs.last_hidden_state
        cls_token = last_hidden_state[:, 0, :]
        cls_token = self.dropout(cls_token)
        logits = self.regressor(cls_token)
        return logits


num_outputs = 2
num_inputs = 256

# Initialize the BERT-based classification model
bert_regressor = BERTRegressor(model, num_inputs, num_outputs).to(device)

# Define the optimizer and the loss function
optimizer = torch.optim.Adam(bert_regressor.parameters(), lr=2e-4)

# %%
# Define model arguments
# Train the model
num_epochs = 10
batch_size = 64

# set a timer to see how long it takes to train the model
start_time = time.time()
for epoch in range(num_epochs):
    for i in range(0, len(train_input_ids), batch_size):
        input_ids_batch = train_input_ids[i:i+batch_size].to(device)
        attention_mask_batch = torch.ones_like(input_ids_batch).to(device)
        region_ids_batch = train_labels[i:i+batch_size].to(device)

        bert_regressor.zero_grad()

        logits = bert_regressor(
            input_ids_batch, attention_mask_batch)
        loss = loss_fn(logits, region_ids_batch)

        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set after each epoch
    with torch.no_grad():
        test_distances = []
        for i in range(0, len(test_input_ids), batch_size):
            test_input_ids_batch = test_input_ids[i:i+batch_size].to(device)
            test_attention_mask_batch = test_attention_mask[i:i+batch_size].to(device)
            test_labels_batch = test_labels[i:i+batch_size].to(device)

            test_logits_batch = bert_regressor(
                test_input_ids_batch, test_attention_mask_batch)
            test_distance_batch = haversine_distance(
                test_logits_batch[:, 0], test_logits_batch[:, 1],
                test_labels_batch[:, 0], test_labels_batch[:, 1]
            ).mean()
            test_distances.append(test_distance_batch.item())

        test_distance = np.mean(test_distances)
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train loss: {loss}")
        print(f"Test distance: {test_distance}")
        print()


end_time = time.time()
print(f"Training time: {end_time - start_time}")


predicted_coords = logits.tolist()
# %%
torch.save(bert_regressor.state_dict(),
           'electra_regressor.pth')

# save to output file
output_file = 'predicted_labels.csv'

with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    for coord in predicted_coords:
        writer.writerow([coord])
# %%

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading data...
Time taken to load data: 87.28222823143005 seconds
Epoch 1/10
Train loss: 1324.9549560546875
Test distance: 1394.2897694618887

Epoch 2/10
Train loss: 1346.6314697265625
Test distance: 1392.4662002167815

Epoch 3/10
Train loss: 1324.2744140625
Test distance: 1392.2947684957855

Epoch 4/10
Train loss: 1348.8629150390625
Test distance: 1391.5841693383597

Epoch 5/10
Train loss: 1312.757568359375
Test distance: 1392.0720997910355

Epoch 6/10
Train loss: 1318.3541259765625
Test distance: 1392.0195412275623

Epoch 7/10
Train loss: 1370.1702880859375
Test distance: 1393.0411453333084

Epoch 8/10
Train loss: 1355.2144775390625
Test distance: 1392.28341747056

Epoch 9/10
Train loss: 1296.55712890625
Test distance: 1391.8092479340253

Epoch 10/10
Train loss: 1334.9927978515625
Test distance: 1392.3621650016294

Training time: 14223.342560768127
